In [1]:
import pandas as pd

In [4]:
# data/253425.csvを読み込む
# 1行目はファンド名、2行目がヘッダー、3行目からデータ
# 文字コードはShift_JIS
df = pd.read_csv('data/253425.csv', encoding='shift_jis', header=1)

In [20]:
# 基準日、期間（週）を指定すると、その期間のリターンを計算する関数
def calculate_return(df, base_date, period_weeks):
    # '基準日'列をdatetime型に変換（既に変換済みの場合は不要）
    df['基準日'] = pd.to_datetime(df['基準日'])
    base_date_dt = pd.to_datetime(base_date)
    # 基準日からperiod_weeks週間前の日付（指定された開始日）を計算
    start_date_requested = base_date_dt - pd.Timedelta(weeks=period_weeks)
    
    # 指定日以下で最大の日付を使用
    base_date_used = df.loc[df['基準日'] <= base_date_dt, '基準日'].max()
    start_date_used = df.loc[df['基準日'] <= start_date_requested, '基準日'].max()
    
    if pd.isnull(base_date_used) or pd.isnull(start_date_used):
        raise ValueError("データ内に有効な日付が存在しません")
    
    # 基準日と開始日に対応する基準価額（分配金再投資）の取得
    base_price_arr = df.loc[df['基準日'] == base_date_used, '基準価額（分配金再投資）(円)']
    start_price_arr = df.loc[df['基準日'] == start_date_used, '基準価額（分配金再投資）(円)']
    
    if base_price_arr.empty or start_price_arr.empty:
        raise ValueError("指定された基準日または開始日に対応する基準価額が存在しません")
    
    base_price = base_price_arr.values[0]
    start_price = start_price_arr.values[0]
    
    # リターンを計算（%で小数点以下2桁まで）
    return round((base_price - start_price) / start_price * 100, 2)

In [19]:
base_date = '2025-09-19'
period_weeks = 156
start_date = pd.to_datetime(base_date) - pd.Timedelta(weeks=period_weeks)

In [11]:
df.head

<bound method NDFrame.head of              基準日  基準価額(円)  基準価額（分配金再投資）(円)  分配金（税引前）(円)  純資産総額（億円）
0     2018/10/31    10000            10000          NaN       0.10
1     2018/11/01    10077            10077          NaN       0.10
2     2018/11/02    10209            10209          NaN       0.11
3     2018/11/05    10228            10228          NaN       0.35
4     2018/11/06    10259            10259          NaN       0.86
...          ...      ...              ...          ...        ...
1676  2025/09/16    29995            29995          NaN   74290.49
1677  2025/09/17    29794            29794          NaN   73874.72
1678  2025/09/18    29864            29864          NaN   74138.55
1679  2025/09/19    30174            30174          NaN   74967.33
1680  2025/09/22    30296            30296          NaN   75312.18

[1681 rows x 5 columns]>

In [21]:
print(calculate_return(df, '2025-09-19', 26))
print(calculate_return(df, '2025-09-19', 52))
print(calculate_return(df, '2025-09-19', 156))
print(calculate_return(df, '2025-09-19', 260))

16.34
22.71
80.35
171.25
